In [166]:
import tweepy  #another popular twitter API wrapper
import json
import datetime
import itertools, nltk, string
import gensim
import networkx

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

consumer_key = 'rOO0OLpyMeE27NOB0gxnoEwaq'
consumer_secret = 'ohY5H9hg9d3u5GWcmsWyPWj0LwsQaRd0jjBeB75blpgyPuqbej'
oauth_token = '62111881-muSYG05rnrz9DuieFcA9SeHRe8jkKuovO4jw875XY'
oauth_token_secret = 'UTN3NYCNfbK61ezvO7a06Kj7A5myePJOAM9ujrxG2yuJV'

# the authentication process for tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

auth.set_access_token(oauth_token, oauth_token_secret)

api = tweepy.API(auth)

print(api)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\skena\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\skena\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\skena\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [167]:
text = ''
page_count = 0
for page in tweepy.Cursor(api.user_timeline, id="Flipkart", tweet_mode="extended").pages():
    # page is a list of statuses
    while 1==1:
        try:
            tw = page.pop()
            if (tw.in_reply_to_status_id == None) and (tw.author.name == 'Flipkart'):
                tweettext = str( tw.full_text.encode('ascii',errors='ignore'))
                if tweettext.startswith("rt @") == True:
                    tweettext = tw.retweeted_status.full_text
                text= text + tweettext + '. '
        except IndexError:
            break;
            
    page_count = page_count + 1
    
    if page_count > 10:
        break
        
    


In [168]:
text
import pandas as pd

f= open("word_text.txt","w+", encoding='utf-8')
f.write(text)
f.close() 

In [169]:
# Noun Phrase candidate

In [170]:
def extract_candidate_chunks(text, grammar=r'KT: {(<JJ>* <NN.*>+ <IN>)? <JJ>* <NN.*>+}'):
    import itertools, nltk, string
    
    def lambda_unpack(f):
        return lambda args: f(*args)
    
    # exclude candidates that are stop words or entirely punctuation
    punct = set(string.punctuation)
    stop_words = set(nltk.corpus.stopwords.words('english'))
    # tokenize, POS-tag, and chunk using regular expressions
    chunker = nltk.chunk.regexp.RegexpParser(grammar) # punkt
    tagged_sents = [nltk.pos_tag(nltk.word_tokenize(sent)) for sent in nltk.sent_tokenize(text)] # averaged_perceptron_tagger
    
    all_chunks = list(itertools.chain.from_iterable(nltk.chunk.tree2conlltags(chunker.parse(tagged_sent))
                                                    for tagged_sent in tagged_sents))
    # join constituent chunk words into a single chunked phrase
    candidates = [' '.join(word for word, pos, chunk in group).lower() for key, group in
                itertools.groupby(all_chunks, lambda_unpack(lambda word, pos, chunk: chunk != 'O')) if key]

    return [cand for cand in candidates
            if cand not in stop_words and not all(char in punct for char in cand)]

In [118]:
extract_candidate_chunks('This is a car. And I am driving it. This red car is good')

['car', 'red car']

In [171]:
def extract_candidate_words(text, good_tags=set(['JJ','JJR','JJS','NN','NNP','NNS','NNPS'])):
    import itertools, nltk, string

    # exclude candidates that are stop words or entirely punctuation
    punct = set(string.punctuation)
    stop_words = set(nltk.corpus.stopwords.words('english'))
    # tokenize and POS-tag words
    tagged_words = itertools.chain.from_iterable([nltk.pos_tag(nltk.word_tokenize(sent)) for sent in nltk.sent_tokenize(text)])
    # filter on certain POS tags and lowercase all words
    candidates = [word.lower() for word, tag in tagged_words
                  if tag in good_tags and word.lower() not in stop_words
                  and not all(char in punct for char in word)]

    return candidates

extract_candidate_words('This is a car. And I am driving it.')

['car']

In [172]:
def score_keyphrases_by_textrank(text, n_keywords=0.05):
    from itertools import takewhile, tee
    import networkx, nltk
    
    # tokenize for all words, and extract *candidate* words
    words = [word.lower()
             for sent in nltk.sent_tokenize(text)
             for word in nltk.word_tokenize(sent)]
    candidates = extract_candidate_words(text)
    # build graph, each node is a unique candidate
    graph = networkx.Graph()
    graph.add_nodes_from(set(candidates))
    # iterate over word-pairs, add unweighted edges into graph
    def pairwise(iterable):
        """s -> (s0,s1), (s1,s2), (s2, s3), ..."""
        a, b = tee(iterable)
        next(b, None)
        return zip(a, b)
    # add the edge for pairwise
    for w1, w2 in pairwise(candidates):
        if w2:
            graph.add_edge(*sorted([w1, w2]))
    # score nodes using default pagerank algorithm, sort by score, keep top n_keywords
    ranks = networkx.pagerank(graph)
    # Percentage candidate words to be taken
    if 0 < n_keywords < 1:
        n_keywords = int(round(len(candidates) * n_keywords))
    word_ranks = {word_rank[0]: word_rank[1]
                  for word_rank in sorted(ranks.items(), key=lambda x: x[1], reverse=True)[:n_keywords]}
    keywords = set(word_ranks.keys())
    # merge keywords into keyphrases
    keyphrases = {}
    j = 0
    # Create phrase
    for i, word in enumerate(words):
        if i < j:
            continue
        if word in keywords:
            kp_words = list(takewhile(lambda x: x in keywords, words[i:i+10]))
            avg_pagerank = sum(word_ranks[w] for w in kp_words) / float(len(kp_words))
            keyphrases[' '.join(kp_words)] = avg_pagerank
            # counter as hackish way to ensure merged keyphrases are non-overlapping
            j = i + len(kp_words)
    
    return sorted(keyphrases.items(), key=lambda x: x[1], reverse=True)

In [174]:
score_keyphrases_by_textrank(text, 0.05)

[('flipkart', 0.02679198592853726),
 ('https', 0.026106697498769496),
 ('b', 0.020157777019431396),
 ('flipkart beauty', 0.014771037864212515),
 ('flipkartpoll https', 0.014735681783379157),
 ('flipkart fashion', 0.01434511998472549),
 ('flipkart summer', 0.014333995126510096),
 ('flipkart gift', 0.014215343146262225),
 ('flipkart games', 0.014144878562073148),
 ('flipkart ceo', 0.01410765149862175),
 ('harbudgetmeinquality https', 0.013997469795136847),
 ("b'rt", 0.013555805668697336),
 ('flipkart fashion daily', 0.010285264256142223),
 ('amp', 0.009374377241811253),
 ('best', 0.005563743702608252),
 ('best way', 0.00447922589612437),
 ('shop', 0.00415171916439844),
 ('day', 0.004136115698079278),
 ('india', 0.004088889841113604),
 ('time', 0.004080524862356444),
 ('new', 0.004018223039174103),
 ('latest', 0.0038438128740273104),
 ('best cricket', 0.0037869076251618953),
 ('something new', 0.0037710228778331857),
 ('best quality', 0.003769940521781299),
 ('best things', 0.003680848033

## Text Rank line by line